In [1]:
import os
from tqdm import tqdm
import warnings, transformers, logging, torch
import pandas as pd
import numpy as np
import shutil
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification

/home/bhavik/projects/kaggle-patent-phrase-matching/venv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
os.environ["WANDB_DISABLED"] = "true"
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [4]:
class CFG:
    if iskaggle:
        input_path = '../input/us-patent-phrase-to-phrase-matching'
    else:
        input_path = '/home/bhavik/projects/kaggle-patent-phrase-matching/data'
    
    model_path = 'anferico/bert-for-patents'
    # model_path = 'microsoft/deberta-v3-base'
    
    learning_rate = 1e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 6
    batch_size = 16

    max_len=133

In [5]:
train_df = pd.read_csv(f"{CFG.input_path}/train.csv")
titles = pd.read_csv(f"{CFG.input_path}/titles.csv")
train_df = train_df.merge(titles, left_on='context', right_on='code')

In [6]:
train_df.head(25)

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
5,067203128142739c,abatement,greenhouse gases,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
6,061d17f04be2d1cf,abatement,increased rate,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
7,e1f44e48399a2027,abatement,measurement level,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
8,0a425937a3e86d10,abatement,minimising sounds,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
9,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    CFG.model_path,
    additional_special_tokens = ['[abstract]','[claim]','[summary]','[invention]','[cpc]']
)

In [8]:
# train_df['input'] = train_df.anchor + '[cpc]' + train_df.title + '[SEP]' + train_df.target  # bert patents
train_df['input'] = train_df.anchor + '[SEP]' + train_df.target + '[cpc]' + train_df.title  # bert patents (best val score so far)
# train_df['input'] = train_df['title'] + ' ' + train_df['anchor']  # deberta
# train_df['input'] = train_df.title + '[SEP]' + train_df.anchor + '[SEP]' + train_df.target # deberta

Bert for patents has special tokens that can be used to map the input to special parts of a patent. Could map to following input structures:

[CLS]anchor[cpc]context_text[SEP]target[SEP]

or

[CLS]anchor[SEP]target[cpc]context_text[SEP]

In [9]:
# ====================================================
# Define max_len
# ====================================================
lenghts = []
for input_text in train_df['input']:
    lenghts.append(len(tokenizer(input_text, add_special_tokens=False)['input_ids']))
    
max_len = max(lenghts) + 4 # CLS + SEP + SEP + SEP
CFG.max_len = max_len
max_len

70

In [10]:
anchors = train_df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)
anchors[:5]

array(['respective emitter', 'dental aligners', 'committee',
       'form trench isolation', 'pulping apparatus'], dtype=object)

In [11]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [12]:
is_val = np.isin(train_df.anchor, val_anchors)
idxs = np.arange(len(train_df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs), len(trn_idxs)

(9418, 27055)

In [13]:
val_df = train_df[is_val]
len(val_df)

9418

In [14]:
training_df = train_df[~is_val]
len(training_df)

27055

In [15]:
training_df.score.mean(), val_df.score.mean()

(0.35914803178710036, 0.37043427479294966)

In [16]:
def corr(eval_pred): 
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [17]:
encoded = tokenizer(training_df.iloc[0]['input'])

In [18]:
tokenizer.convert_ids_to_tokens(encoded['input_ids'])

['[CLS]',
 'abatement',
 '[SEP]',
 'abatement',
 'of',
 'pollution',
 '[cpc]',
 'furniture',
 ';',
 'domestic',
 'articles',
 'or',
 'appliances',
 ';',
 'coffee',
 'mills',
 ';',
 'spice',
 'mills',
 ';',
 'suction',
 'cleaners',
 'in',
 'general',
 '[SEP]']

In [19]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        label = self.label[item]
        
        return {
        **tokenizer(inputs, max_length=CFG.max_len, padding="max_length"),
        #**tokenizer(inputs),  # use for bert-patents
        # **tokenizer(inputs, targets),
        'label': label.astype(np.float32)
    }

In [20]:
train_dataset = TrainDataset(training_df)
val_dataset = TrainDataset(val_df)

In [21]:
args = TrainingArguments(
    'outputs',
    save_strategy='no',
    learning_rate=CFG.learning_rate, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    fp16=True,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=CFG.batch_size, 
    per_device_eval_batch_size=CFG.batch_size*2,
    num_train_epochs=CFG.epochs, 
    weight_decay=CFG.weight_decay, 
    report_to='none'
)
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
model_trainer = Trainer(
    model, 
    args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,           
    tokenizer=tokenizer, 
    compute_metrics=corr,
)

In [22]:
model_trainer.train()

  5%|▍         | 501/10146 [01:33<30:11,  5.32it/s]

{'loss': 0.1195, 'learning_rate': 4.9064039408867e-06, 'epoch': 0.3}


 10%|▉         | 1001/10146 [03:11<29:32,  5.16it/s]

{'loss': 0.0624, 'learning_rate': 9.832512315270937e-06, 'epoch': 0.59}


 15%|█▍        | 1501/10146 [04:47<27:40,  5.21it/s]

...

 99%|█████████▊| 10001/10146 [34:29<00:28,  5.13it/s]

{'loss': 0.0069, 'learning_rate': 6.568716076514325e-09, 'epoch': 5.91}


100%|██████████| 10146/10146 [35:19<00:00,  4.79it/s]

{'eval_loss': 0.023296654224395752, 'eval_pearson': 0.8235458416009239, 'eval_runtime': 21.6038, 'eval_samples_per_second': 435.941, 'eval_steps_per_second': 13.655, 'epoch': 6.0}
{'train_runtime': 2119.3048, 'train_samples_per_second': 76.596, 'train_steps_per_second': 4.787, 'train_loss': 0.024693631124806645, 'epoch': 6.0}


TrainOutput(global_step=10146, training_loss=0.024693631124806645, metrics={'train_runtime': 2119.3048, 'train_samples_per_second': 76.596, 'train_steps_per_second': 4.787, 'train_loss': 0.024693631124806645, 'epoch': 6.0})

In [23]:
shutil.rmtree('outputs')